# Analytic Functions

Analytic functions allow us to perform complex calculations with relatively straightforward syntax. For instance, we can quickly calculate moving averages and running totals, among other quantities.

To understand how to write analytic functions, we'll work with a small table containing data from two different people who are training for a race. The id column identifies each runner, the date column holds the day of the training session, and time shows the time (in minutes) that the runner dedicated to training. Say we'd like to calculate a moving average of the training times for each runner, where we always take the average of the current and previous training sessions.

![https://i.imgur.com/rehp8HM.png](https://i.imgur.com/rehp8HM.png)

All analytic functions have an `OVER` clause, which defines the sets of rows used in each calculation. The `OVER` clause has three (optional) parts:

1. The `PARTITION BY` clause divides the rows of the table into different groups. In the query above, we divide by `id` so that the calculations are separated by runner.
2. The `ORDER BY` clause defines an ordering within each partition. In the sample query, ordering by the `date` column ensures that earlier training sessions appear first.
3. The final clause (`ROWS BETWEEN 1 PRECEDING AND CURRENT ROW`) is known as a **window frame clause**. It identifies the set of rows used in each calculation. We can refer to this group of rows as a **window**. (Actually, analytic functions are sometimes referred to as **analytic window functions** or simply **window functions**!) 

![https://i.imgur.com/GjiKlA7.png](https://i.imgur.com/GjiKlA7.png)

## (More on) window frame clauses

(More on) window frame clauses:

1. `ROWS BETWEEN 1 PRECEDING AND CURRENT ROW` - the previous row and the current row.
2. `ROWS BETWEEN 3 PRECEDING AND 1 FOLLOWING` - the 3 previous rows, the current row, and the following row. 
3. `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` - all rows in the partition.

## Three types of analytic functions

### 1) Analytic aggregate functions

As you might recall, `AVG()` (from the example above) is an aggregate function. The `OVER` clause is what ensures that it's treated as an analytic (aggregate) function. **Aggregate functions** take all of the values within the window as input and return a single value.

1. `MIN()` (or `MAX()`) - Returns the minimum (or maximum) of input values
2. `AVG()` (or `SUM()`) - Returns the average (or sum) of input values 
3. `COUNT()` - Returns the number of rows in the input

### 2) Analytic navigation functions

**Navigation functions** assign a value based on the value in a (usually) different row than the current row.

1. `FIRST_VALUE()` (or `LAST_VALUE()`) - Returns the first (or last) value in the input
2. `LEAD()` (and `LAG()`) - Returns the value on a subsequent (or preceding) row

### 3) Analytic numbering functions

**Numbering functions** assign integer values to each row based on the ordering.

1. `ROW_NUMBER()` - Returns the order in which rows appear in the input (starting with `1`)
2. `RANK()` - All rows with the same value in the ordering column receive the same rank value, where the next row receives a rank value which increments by the number of rows with the previous rank value.

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
table_ref = dataset_ref.table("bikeshare_trips")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

Each row of the table corresponds to a different bike trip, and we can use an analytic function to calculate the cumulative number of trips for each date in 2015.

In [2]:
# Query to count the (cumulative) number of trips per day
num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                      FROM trips_by_day
                  """

# Run the query, and return a pandas DataFrame
num_trips_result = client.query(num_trips_query).result().to_dataframe()
num_trips_result.head()

,trip_date,num_trips,cumulative_trips
0,2015-01-01,181,181
1,2015-01-02,428,609
2,2015-01-03,283,892
3,2015-01-04,206,1098
4,2015-01-05,1186,2284


The query uses a **common table expression** (**CTE**) to first calculate the daily number of trips. Then, we use `SUM()` as an aggregate function.

1. Since there is no `PARTITION BY` clause, the entire table is treated as a single partition.
2. The `ORDER BY` clause orders the rows by date, where earlier dates appear first. 
3. By setting the window frame clause to `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`, we ensure that all rows up to and including the current date are used to calculate the (cumulative) sum. (Note: If you read the documentation, you'll see that this is the default behavior, and so the query would return the same result if we left out this window frame clause.)

The next query tracks the stations where each bike began (in `start_station_id`) and ended (in `end_station_id`) the day on October 25, 2015.

In [3]:
# Query to track beginning and ending stations on October 25, 2015, for each bike
start_end_query = """
                  SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                      start_station_id,
                      end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25' 
                  """

# Run the query, and return a pandas DataFrame
start_end_result = client.query(start_end_query).result().to_dataframe()
start_end_result.head()

,bike_number,trip_time,first_station_id,last_station_id,start_station_id,end_station_id
0,22,13:25:00,2,16,2,16
1,25,11:43:00,77,51,77,60
2,25,12:14:00,77,51,60,51
3,29,14:59:00,46,74,46,60
4,29,21:23:00,46,74,60,74


The query uses both `FIRST_VALUE()` and `LAST_VALUE()` as analytic functions.

1. The `PARTITION BY` clause breaks the data into partitions based on the bike_number column. Since this column holds unique identifiers for the bikes, this ensures the calculations are performed separately for each bike.
2. The `ORDER BY` clause puts the rows within each partition in chronological order.
3. Since the window frame clause is `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`, for each row, its entire partition is used to perform the calculation. (This ensures the calculated values for rows in the same partition are identical.)

## Exercise

In [1]:
from google.cloud import bigquery
client = bigquery.Client()
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
table_ref = dataset_ref.table("taxi_trips")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,6f15e327be1e4a6668c07501b9793ee88f144cee,69e42e59ac0a3ab4a2e277aa345e5a5d9e62b6748e7d8a...,2015-06-15 10:45:00+00:00,2015-06-15 10:45:00+00:00,0,0.0,None,None,None,None,...,0.0,7.74,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
1,ab006693a83d513cf05c5071c21e34b07e179ca2,996b81064eed2e1e1e8fd1fcafe6ea315cc5190ad90d88...,2015-04-04 19:00:00+00:00,2015-04-04 19:00:00+00:00,0,0.0,None,None,None,None,...,0.0,10.00,Credit Card,Suburban Dispatch LLC,None,None,None,None,None,None
2,a1c02a88359a6e64d1aa7e882fc7daba514677b7,d0cf86bf92d34b655a1826a727666790817bad226691cd...,2015-04-25 10:45:00+00:00,2015-04-25 10:45:00+00:00,0,0.0,None,None,None,None,...,0.0,7.31,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
3,36873b6356be4604b935776ab7c37ddef17a9894,609fa4bfe241aacb61e59c5b52abcbbd467b8e71d53dd9...,2015-04-18 20:30:00+00:00,2015-04-18 21:00:00+00:00,1380,0.0,None,None,None,None,...,0.0,14.65,Cash,Choice Taxi Association,None,None,None,None,None,None
4,077f1eb1512cbbb5a64bcc815e416f3df234c690,25e9bbb9704eb82311a4d4036baba6451cd27138c4d6eb...,2015-03-31 10:00:00+00:00,2015-03-31 10:00:00+00:00,0,0.0,None,None,None,None,...,0.0,10.05,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None


### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis. Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips. Amend the (partial) query below to return a DataFrame with two columns:

1. `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
2. `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame. For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the avg_num_trips column. Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out this tutorial in the Intro to SQL micro-course.

In [2]:
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                          SELECT DATE(trip_start_timestamp) AS trip_date,
                              COUNT(*) as num_trips
                          FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                          WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                          GROUP BY trip_date
                          ORDER BY trip_date
                      )
                      SELECT trip_date, 
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas. So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2. Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the `RANK()` function.


In [3]:
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() 
                            OVER (
                               PARTITION BY pickup_community_area
                               ORDER BY trip_start_timestamp
                            ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`.

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip). Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`). Take the time now to make sure that the values in the `prev_break` column make sense to you!

In [4]:
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1)
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """